In [1]:
F = {'n': 5,
     'clauses': [
                  [2, -5],
                  [1, 5],
                  [-3, 4],
                  [-2, -4],
                  [3, 4]
                ]
    }

F_ = {'n': 5,
      'clauses': F['clauses'] + [[2, -4]]}

In [2]:
# Building the implication graph
def build(F):
    graph = {}
    V = []  # The set of nodes
    for P in range(1, F['n'] + 1):
        V.append(P)
        V.append(-P)

    E = []  # The set of edges
    for clause in F['clauses']: # We assume all clauses have exactly 2 literals
        edge1 = (-clause[0], clause[1])
        edge2 = (-clause[1], clause[0])
        E.append(edge1)
        E.append(edge2)

    graph['V'] = V
    graph['E'] = E

    return graph

In [3]:
# Function to check whether there is a path from A to B
def path(graph, A, B):
    E = graph['E']

    visited = [A]

    while True:
        old_visited_length = len(visited)
        for u, v in E:
            if u in visited and v not in visited:
                visited.append(v)

        if B in visited:
            return True

        if old_visited_length == len(visited):
            break
        
    return False

In [4]:
# Function to check whether the graph is consistent or not
def check_consistent(F):
    graph = build(F)
    for i in range(1, F['n'] + 1):
        if path(graph, i, -i) and path(graph, -i, i):
            return False
    return True

In [5]:
print(check_consistent(F))
print(check_consistent(F_))

True
False


In [9]:
def two_sat_algorithm(F):
    n = F['n']
    graph = build(F)
    edges = graph['E']

    if not check_consistent(F):
        return "The implication graph is not consistent. This formula has no solution."

    # Initialization
    P = {i: -1 for i in range(1, n+1)}

    while True:
        unassigned_found = False

        # Picking L such that L is unassigned and there is no path from L to not(L)
        for i in range(1, n+1):
            if P[i] == -1:
                unassigned_found = True
                if not path(graph, i, -i):
                    # We found L, so we set it to 1
                    P[i] = 1
                else:
                    # In this case, we take L = - P[i]
                    P[i] = 0
                break

        # If no unassigned variable was found, terminate
        if not unassigned_found:
            return P
        
        while True:
            found = False
            for L1, L2 in edges:
                if L1 > 0 and L2 > 0:
                    if val(L1, P) == 1 and val(L2, P) == -1:
                        found = True
                        # Set L2 to 1
                        P[abs(L2)] = 1 if L2 > 0 else 0

            if not found:
                # There is no edge 1 => -1 remaining
                break


def val(L, P):
    if L > 0:
        return P[L]
    else:
        return not(P[-L])

In [10]:
two_sat_algorithm(F)

{1: 1, 2: 0, 3: 1, 4: 1, 5: 0}

In [8]:
two_sat_algorithm(F_)

'The implication graph is not consistent. This formula has no solution.'